# Global setup and package installation used in most phases

## Colab + GPU Detection Utilities

In [ ]:
import subprocess

def is_running_in_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

def get_available_gpu_memory_gb():
    try:
        output = subprocess.check_output(
            ["nvidia-smi", "--query-gpu=memory.free", "--format=csv,nounits,noheader"],
            encoding="utf-8"
        )
        free_mem_mb = int(output.strip().split("\n")[0])
        return free_mem_mb / 1024
    except Exception:
        return 0.0


## install dependencies

In [ ]:
if is_running_in_colab():
    # Install the required packages
    !pip install kagglehub pandas
    !pip install -q transformers accelerate bitsandbytes sentencepiece pydantic huggingface_hub xformers
    !pip install regex json5
else:
    %pip install kagglehub pandas
    %pip install -q transformers accelerate sentencepiece pydantic huggingface_hub xformers
    #%pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cu121
    #%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
    %pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu128
    %pip install -U bitsandbytes
    %pip install regex json5


## Login to huggingface

In [ ]:
from huggingface_hub import login
import os

# Set your token here securely or prompt for it in Colab
# Recommended: store in Colab secrets or environment variable
HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN")


if not HF_TOKEN:
    if is_running_in_colab():
        # If running in Colab, use the Colab secrets
        try:
            from google.colab import userdata
            HF_TOKEN = userdata.get('HF_TOKEN')
            if not HF_TOKEN:
                raise ValueError("⚠️ Hugging Face token not found in Colab secrets.")
            print("🔑 Hugging Face token found in Colab secrets.")
        except ImportError:
            print("⚠️ Unable to authenticate in Colab. Please set your Hugging Face token manually.")
    else:
        # Prompt for token if not set in environment
        print("🔑 Please enter your Hugging Face token:")
        # For Colab or local prompt input
        HF_TOKEN = input("🔑 Enter your Hugging Face token: ").strip()

login(token=HF_TOKEN)


## Setup Kaggle Credentials

In [ ]:
import shutil

def setup_kaggle_credentials():
    kaggle_path = os.path.expanduser('~/.kaggle/kaggle.json')
    if not os.path.exists(kaggle_path):
        from google.colab import files
        print("📂 Upload kaggle.json file...")
        uploaded = files.upload()
        os.makedirs(os.path.dirname(kaggle_path), exist_ok=True)
        for filename in uploaded.keys():
            shutil.move(filename, kaggle_path)
        os.chmod(kaggle_path, 0o600)
        print(f"✅ Kaggle credentials setup at {kaggle_path}")
    else:
        print(f"✅ Kaggle credentials already exist at {kaggle_path}")

setup_kaggle_credentials()

## Mount Google Drive (Colab)

In [ ]:
if is_running_in_colab():
   from google.colab import drive
   drive.mount('/content/drive')

##  Load Mistral-Instruct with Fallback to Quantized

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

def load_model_pipeline(model_name: str, hf_token: str):
    has_cuda = torch.cuda.is_available()
    free_mem = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3) if has_cuda else 0
    print(f"💻 CUDA: {has_cuda} | GPU Memory: {free_mem:.2f} GB")

    device_map = {"": 0} if has_cuda else "cpu"
    use_4bit = has_cuda and free_mem < 24

    # Set quantization config
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True if use_4bit else False,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4"
    ) if use_4bit else None

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token  # ✅ Fix warning about pad_token

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map=device_map,
        quantization_config=quant_config,
        torch_dtype=torch.float16 if not quant_config else None,
        trust_remote_code=True,
        token=hf_token
    )

    print(f"✅ Model loaded on {next(model.parameters()).device}")
    return pipeline("text-generation", model=model, tokenizer=tokenizer, batch_size=1)


In [ ]:
llm_pipeline = load_model_pipeline(
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    hf_token=HF_TOKEN
)


# Global utilities

### Utility to merge normalized json files

In [ ]:
import json
import shutil
from pathlib import Path

def merge_json_files(
    source_dir: Path,
    output_file: Path,
    pattern: str,
    merged_dir: Path
):
    source_dir.mkdir(parents=True, exist_ok=True)
    merged_dir.mkdir(parents=True, exist_ok=True)

    merged_data = []

    # Load existing output if it exists
    if output_file.exists():
        with open(output_file, "r", encoding="utf-8") as f:
            try:
                merged_data = json.load(f)
            except json.JSONDecodeError:
                print(f"⚠️ Could not decode {output_file}, starting from scratch.")

    # Identify matching files
    files_to_merge = sorted(source_dir.glob(pattern))

    for file_path in files_to_merge:
        with open(file_path, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
                if isinstance(data, list):
                    merged_data.extend(data)
                else:
                    print(f"⚠️ Skipping {file_path.name}: not a list.")
            except Exception as e:
                print(f"⚠️ Failed to parse {file_path.name}: {e}")
                continue

        # Move to merged folder
        shutil.move(str(file_path), merged_dir / file_path.name)
        print(f"✅ Merged and moved: {file_path.name}")

    # Write combined output
    if merged_data:
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(merged_data, f, indent=2)
        print(f"💾 Saved to: {output_file}")
    else:
        print("📭 No valid data to merge.")




### Infer JD Domains

In [ ]:
domain_keywords_dict = {
    'advocate': ['advocate'],
    'agriculture': ['agriculture'],
    'apparel': ['apparel'],
    'arts': ['arts'],
    'automobile': ['automobile'],
    'aviation': ['aviation'],
    'banking': ['banking'],
    'bpo': ['bpo'],
    'business development': ['business', 'development', 'business development', 'business-development'],
    'chef': ['chef'],
    'construction': ['construction'],
    'consultant': ['consultant'],
    'data scientist': ['data', 'data analyst', 'data scientist', 'scientist'],
    'designing': ['designing', 'designer'],
    'digital media': ['digital', 'digital marketing executive', 'media', 'digital media', 'digital-media'],
    'engineering': ['engineering'],
    'finance': ['finance', 'financial analyst'],
    'healthcare': ['healthcare'],
    'hr': ['hr'],
    'information technology': ['information', 'technology', 'information technology', 'information-technology'],
    'public relations': ['public', 'relations', 'public relations', 'public-relations'],
    'marketing': ['marketing'],
    'sales': ['sales', 'sales executive'],
    'teacher': ['teacher'],
    'technician': ['technician'],
    'training': ['training'],
    'web designing': ['web', 'designing'],
    'fitness': ['fitness'],
    'accountant': ['accountant', 'accounting']
}


In [ ]:
def infer_domain_from_title(title):
    title_lower = title.lower()
    for domain, keywords in domain_keywords_dict.items():
        if any(kw in title_lower for kw in keywords):
            return domain
    return "unknown"


In [ ]:
import re
# no longer being used
def infer_job_domain_llm(description: str) -> str:
    print("🔍 Inferring job domain using LLM...")

    prompt = f"""
Given a job description, return only the most likely domain of the job as a one-word or short noun phrase. Do not include explanation, punctuation, or label. Just return the domain.

Job Description:
{description.strip()}

Domain:"""

    try:
        response = llm_pipeline(prompt)[0]['generated_text']

        # Extract only the part after the final "Domain:" (handles echo case)
        if "Domain:" in response:
            response = response.split("Domain:")[-1]

        # Get the first non-empty word/line
        domain = next((line.strip().lower() for line in response.strip().splitlines() if line.strip()), "unknown")

        # Clean up unwanted characters
        domain = re.sub(r"[^a-zA-Z &\-]", "", domain)

        return domain if domain else "unknown"

    except Exception as e:
        print(f"⚠️ LLM inference failed: {e}")
        return "unknown"


### Configurations  

In [ ]:
# ==============================
# 🛠 CONFIGURATION
# ==============================

class Config:
    DATASET_DOWNLOAD_DIR = "datasets"
    JSON_OUTPUT_DIR = "json_outputs_run3"
    JSON_OUTPUT_NORMALIZED_DIR = "json_outputs_run3/normalized"
    AUTO_CLEANUP = True


## Utility to save json to a folder

In [ ]:
import json
import os
# 📦 Save JSON Output with Safety
def save_json_output(data, output_path: str, indent: int = 4, overwrite: bool = True):
    output_dir = os.path.dirname(output_path)
    os.makedirs(output_dir, exist_ok=True)

    if os.path.exists(output_path):
        if overwrite:
            os.remove(output_path)
        else:
            raise FileExistsError(f"File {output_path} already exists and overwrite=False.")

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=indent, ensure_ascii=False)

    print(f"✅ Saved output to {output_path}")


## Utility to load file

### load_ndjson_file() (for resume/jd input)

In [ ]:
from typing import List


def load_ndjson_file(file_path: Path) -> List[dict]:
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    with open(file_path, 'r', encoding='utf-8') as file:
        return [json.loads(line) for line in file if line.strip()]


### load_json_file() (for checkpoint & metadata)

In [ ]:
def load_json_file(file_path: Path) -> dict:
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)


# Phase 2 -	Parse resume/JD into JSON structured scheme

## Define Pydantic Schemas

In [ ]:
from typing import Optional
from pydantic import BaseModel

class Education(BaseModel):
    degree: Optional[str] = None
    field: Optional[str] = None
    institution: Optional[str] = None
    year: Optional[str] = None
    gpa: Optional[float] = None

class Experience(BaseModel):
    job_title: Optional[str] = None
    company: Optional[str] = None
    start_date: Optional[str] = None
    end_date: Optional[str] = None
    duration_in_months: Optional[int] = None
    description: Optional[List[str]] = None
    
class Certification(BaseModel):
    certification: Optional[str] = None
    date_issued: Optional[str] = None

class Project(BaseModel):
    project_title: Optional[str] = None
    description: Optional[str] = None
    start_date: Optional[str] = None
    end_date: Optional[str] = None
    url: Optional[str] = None
    
class Language(BaseModel):
    language: Optional[str] = None
    proficiency: Optional[str] = None
    
class Other(BaseModel):
    section_name: Optional[str] = None
    content: Optional[str] = None

In [ ]:
from typing import List, Optional
from pydantic import BaseModel
import re


class ResumeSchema(BaseModel):
    resume_id: Optional[str] = None
    summary: Optional[str] = None
    education: Optional[List[Education]] = None
    experience: Optional[List[Experience]] = None
    skills: Optional[List[str]] = None
    certifications: Optional[List[Certification]] = None
    projects: Optional[List[Project]] = None
    languages: Optional[List[Language]] = None
    other: Optional[List[Other]] = None
    total_experience_years: Optional[float] = None

    @classmethod
    def normalize(cls, resume_dict: dict) -> dict:
        resume_dict = dict(resume_dict)

        # Normalize sections
        for key in ["skills", "certifications", "projects", "languages"]:
            if not isinstance(resume_dict.get(key), list):
                resume_dict[key] = []

        # Normalize Experience
        normalized_exp = []
        for item in resume_dict.get("experience", []):
            if not isinstance(item, dict):
                continue
            normalized_exp.append({
                "job_title": item.get("job_title", item.get("title", "")),
                "company": item.get("company", ""),
                "start_date": item.get("start_date", ""),
                "end_date": item.get("end_date", ""),
                "duration_in_months": item.get("duration_in_months", None),
                "description": item.get("description", "")
            })
        resume_dict["experience"] = normalized_exp

        # Normalize Education
        normalized_edu = []
        for item in resume_dict.get("education", []):
            if not isinstance(item, dict):
                continue
            degree = item.get("degree", "")
            field = item.get("field", "")
            if not field:
                match = re.search(r"in\\s+(.+)", degree, flags=re.IGNORECASE)
                field = match.group(1).strip() if match else ""
            year = str(item.get("year", "")) if item.get("year") else ""
            gpa = item.get("gpa", None)
            normalized_edu.append({
                "degree": degree,
                "field": field,
                "institution": item.get("institution", ""),
                "year": year,
                "gpa": gpa
            })
        resume_dict["education"] = normalized_edu

        # Total Experience fallback
        if "total_experience_years" not in resume_dict:
            resume_dict["total_experience_years"] = 0.0

        return resume_dict


In [ ]:
class JobDescriptionSchema(BaseModel):
    jd_id: str
    title: str
    summary: str
    required_experience_years: float
    preferred_degrees: List[str]
    required_skills: List[str]
    optional_skills: List[str]
    certifications: List[str]
    soft_skills: List[str]
    job_location: str
    remote_option: Optional[bool] = False
    employment_type: Optional[str] = None
    inferred_domain: str = "unknown"

    @classmethod
    def normalize(cls, jd_dict: dict) -> dict:
        jd_dict = dict(jd_dict)

        aliases = {
            "years_required": "required_experience_years",
            "requirements": "required_skills",
            "degree_preferences": "preferred_degrees",
            "certs": "certifications",
            "skills_soft": "soft_skills",
            "job_summary": "summary"
        }
        for old, new in aliases.items():
            if old in jd_dict and new not in jd_dict:
                jd_dict[new] = jd_dict.pop(old)

        # Required Experience Extraction
        def extract_experience_years(text: str) -> float:
            if not isinstance(text, str):
                return 0.0
            match = re.search(r'(\\d+(\\.\\d+)?)\\s*\\+?\\s*(years?|yrs?)', text.lower())
            return float(match.group(1)) if match else 0.0

        try:
            val = jd_dict.get("required_experience_years")
            if val is None:
                jd_dict["required_experience_years"] = extract_experience_years(jd_dict.get("summary", ""))
            elif isinstance(val, str):
                jd_dict["required_experience_years"] = float(val.split()[0])
            else:
                jd_dict["required_experience_years"] = float(val)
        except Exception:
            jd_dict["required_experience_years"] = 0.0

        # Normalize fields
        for field in ["preferred_degrees", "required_skills", "optional_skills", "certifications", "soft_skills"]:
            if not isinstance(jd_dict.get(field), list):
                jd_dict[field] = []

        for field in ["title", "summary", "job_location", "employment_type"]:
            jd_dict[field] = jd_dict.get(field, "") or ""

        # Remote Option
        remote_flag = jd_dict.get("remote_option", None)
        if remote_flag is None:
            remote_flag = "remote" in jd_dict.get("summary", "").lower()
        jd_dict["remote_option"] = bool(remote_flag)

        return jd_dict


### Generate schema string

In [ ]:
from typing import get_origin, get_args, Union
from pydantic import BaseModel

def generate_example_structure(model_class) -> dict:
    def default_for_type(field_type):
        origin = get_origin(field_type)
        args = get_args(field_type)

        if origin is list and args:
            inner_type = args[0]
            return [default_for_type(inner_type)]
        elif origin is Union and type(None) in args:
            non_none_types = [arg for arg in args if arg is not type(None)]
            return default_for_type(non_none_types[0]) if non_none_types else ""
        elif field_type is str:
            return ""
        elif field_type in [float, int]:
            return 0.0
        elif isinstance(field_type, type) and issubclass(field_type, BaseModel):
            return generate_example_structure(field_type)
        else:
            return ""

    return {
        field_name: default_for_type(field.annotation)
        for field_name, field in model_class.model_fields.items()
    }


In [ ]:
from functools import lru_cache
import json
from typing import Optional, Type

@lru_cache(maxsize=4)
def get_schema_str(schema_model: Optional[Type]) -> str:
    """
    Returns a cleaned JSON schema string (as example structure) for a given Pydantic model.
    Filters out fields not needed for the prompt like 'resume_id', 'duration_in_months', etc.
    """
    if schema_model is None:
        return "{}"
    
    example = generate_example_structure(schema_model)

    # ⛔️ Fields to remove before rendering into the prompt
    exclude_fields = {"resume_id", "duration_in_months", "total_experience_years"}

    def recursive_filter(obj):
        if isinstance(obj, dict):
            return {
                k: recursive_filter(v)
                for k, v in obj.items()
                if k not in exclude_fields
            }
        elif isinstance(obj, list):
            return [recursive_filter(v) for v in obj]
        return obj

    filtered = recursive_filter(example)

    return json.dumps(filtered, indent=2)


In [ ]:

# ✅ Step 2: Generate schema string from your updated ResumeSchema
schema_str = get_schema_str(ResumeSchema)

# ✅ Step 3: Print and inspect
print(schema_str)


### generate mock sample

In [ ]:
from functools import lru_cache
import json

@lru_cache(maxsize=4)
def get_mock_resume_json() -> str:
    mock_data = {
        "summary": "Certified accountant with 5+ years in financial operations and reporting.",
        "education": [
            {
            "degree": "BBA",
            "field": "Accounting",
            "institution": "State University",
            "year": "2012",
            "gpa": 0.0
            }
        ],
        "experience": [
            {
            "job_title": "Accountant",
            "company": "ABC Corp",
            "start_date": "01/2018",
            "end_date": "12/2020",
            "description": [
                "Prepared financial reports and managed audits.",
                "Assisted in budget forecasting and expense tracking."
            ]
            }
        ],
        "skills": [
            "Excel",
            "SAP",
            "Financial Reporting"
        ],
        "certifications": [
            {
            "certification": "CPA",
            "date_issued": ""
            }
        ],
        "projects": [
            {
            "project_title": "ERP Migration",
            "description": "Led the migration from legacy system to SAP.",
            "start_date": "",
            "end_date": "11/2020",
            "url": ""
            }
        ],
        "languages": [
            {
            "language": "Spanish",
            "proficiency": ""
            }
        ],
        "other": [
            {
            "section_name": "Affiliations",
            "content": "Member of National Association of Accountants"
            }
        ]
    }

    return json.dumps(mock_data, indent=2)


##  Prompt Templates

In [ ]:
RESUME_PROMPT_TEMPLATE = """
[INST]
You are a strict JSON resume parser.

Return only a valid JSON object — no markdown, comments, or extra text.

Follow this example structure (use the same key names and nesting, but do not copy the example values):

{mock_json}

Use this schema (all keys required; do not reorder or remove fields):
{schema}

Rules:
- Use only resume content — do not summarize, infer, or rewrite.
- Copy text as-is with minor cleanup (punctuation/spacing).
- Leave fields empty ("", []) if content is not present in the resume.
- Do not paraphrase or fabricate.
- Limit job descriptions to 10 bullet points max per job, in original order.
- Do not repeat responsibilities across roles.
- Include **all** fields from the schema, even if empty.
- Output must start with `{{` and end with `}}` — nothing before or after.

Resume:
<<<
{text}
>>>
[/INST]
"""


In [ ]:
JD_PROMPT_TEMPLATE = """
You are a JSON job description parser and experience extractor.

Given the following job description text, extract a structured JSON following this schema:

{schema}

Instructions:
- Parse title, summary, skills, certifications, and other fields exactly as shown.
- Pay special attention to "required_experience_years":
    - If experience years are explicitly listed, extract that number.
    - Accept formats like "5+ years", "3-5 years", "8 years required", etc.
    - If multiple ranges are mentioned (e.g., "3-5 years"), use the lower value (3 years).
    - If no years are mentioned explicitly, infer from job title level:
        - "Senior", "Lead" → Assume 5+ years
        - "Mid-level", "Experienced" → Assume 3 years
        - "Entry level", "Junior" → Assume 0-1 years
    - If still ambiguous, default to 0 years.
- Handle remote/hybrid jobs:
    - Set "remote_option" = true if remote keywords are present (remote, work from home, hybrid, WFH).
- Infer the **inferred_domain** from the job description:
    - Return a short domain noun (e.g., "software", "marketing", "data science", "finance", "healthcare").
    - Use the title and summary to guide inference.
    - If uncertain, use "unknown".
- If a field is missing, leave it empty ("") or as an empty list [] depending on the field type.
- Return ONLY a valid JSON object. No extra text, no explanations, no markdown formatting.
- Your output MUST start with a {{.

Job Description Text:
--------------------
{text}
--------------------
"""


##  Inference + Validation Functions

### Generate Raw LLM Output

In [ ]:
def generate_llm_output(prompt: str, max_new_tokens: int = 4096) -> str:
    """Run LLM and return the generated text with token count logging."""
    try:
        # 🔢 Print input token count
        input_tokens = llm_pipeline.tokenizer.encode(prompt, add_special_tokens=False)
        print(f"🧮 Prompt token count: {len(input_tokens)} | Max new tokens: {max_new_tokens} | Estimated total: {len(input_tokens) + max_new_tokens}")

        # 🔁 Generate
        outputs = llm_pipeline(prompt, max_new_tokens=max_new_tokens, do_sample=False)
        
        output_tokens = llm_pipeline.tokenizer.encode(outputs[0]["generated_text"], add_special_tokens=False)
        print(f"📝 Output token count: {len(output_tokens)}")

        return outputs[0]["generated_text"]

    except Exception as e:
        raise RuntimeError(f"LLM generation failed: {e}")


### Sanitize Output: Strip Prompt, Fix Cutoffs

In [ ]:
def sanitize_llm_output(response: str, prompt: str) -> str:
    raw = response.replace(prompt, "").strip()

    # Truncate garbage after the last closing brace
    raw = re.sub(r'}[^}]*$', '}', raw)

    # Remove markdown bullets or --- headers at end
    raw = re.sub(r'(---|•|–|-)\s*$', '', raw, flags=re.MULTILINE)

    return raw


### Regex-based JSON Block Extractor and raw data processor

In [ ]:
import regex
import json5

def extract_json_block(text: str) -> dict:
    """
    Extracts the first valid JSON object from a text using the `regex` module and parses with `json5`.
    This is more robust than standard `json` and can handle trailing commas, comments, etc.
    """
    # Recursive regex pattern to find balanced curly braces (non-greedy)
    pattern = r'(\{(?:[^{}]|(?R))*\})'

    for match in regex.finditer(pattern, text, flags=regex.DOTALL):
        json_candidate = match.group(1)
        try:
            return json5.loads(json_candidate)
        except json5.JSONDecodeError:
            continue

    raise ValueError("❌ No valid JSON object found using regex and json5.")


In [ ]:
def clean_raw_data(raw: str) -> str:
    import regex
    raw = raw.strip().strip("`")

    # Remove code fences
    raw = regex.sub(r"^```(?:json)?\s*|\s*```$", "", raw, flags=regex.IGNORECASE)

    # Extract content between first '{' and last '}'
    start_idx = raw.find('{')
    end_idx = raw.rfind('}')
    if start_idx == -1 or end_idx == -1 or end_idx <= start_idx:
        raise ValueError("Cannot locate complete JSON object in the output")

    json_body = raw[start_idx:end_idx + 1]

    # Fix common issues
    json_body = regex.sub(r",\s*([\]}])", r"\1", json_body)        # remove trailing commas
    json_body = regex.sub(r",\s*,", ",", json_body)                # remove double commas
    json_body = regex.sub(r'"\s*:\s*:', '":', json_body)           # fix "::"
    json_body = regex.sub(r'("\s*:[^,}\]]+")\s*(")', r'\1,\2', json_body)  # fix missing commas

    # Final fallback: force valid quote usage
    json_body = regex.sub(r"‘|’", '"', json_body)
    json_body = regex.sub(r"“|”", '"', json_body)

    return json_body


### Format Resume string for LLM processing

In [ ]:
import regex as re

CANONICAL_HEADER_PATTERNS = [
    r'\b(?:work|professional)?\s*experience\b',
    r'\b(?:education(?:\s+(?:and|&)\s+training)?|training(?:\s+(?:and|&)\s+(?:development|certifications?|programs?)|s)?|specialized\s+training)\b',
    r'\b(?:technical\s+)?skills?\b',
    r'\b(?:certifications?|licenses?|certifications?\s*(?:and|&)\s*licenses?)\b',
    r'\b(?:projects?|key\s+projects|project\s+highlights)\b',
    r'\blanguages?\b',
    r'\b(?:executive\s+)?summary\b',
    r'\b(?:professional\s+)?affiliations?\b',
    r'\b(?:awards?|accomplishments?|honors|achievements)\b',
    r'\bpublications?\b',
    r'\b(?:interests|hobbies|extracurricular\s+activities)\b',
    r'\b(?:volunteer\s+experience|community\s+involvement|volunteering)\b',
    r'\bobjective\b',
    r'\breferences\b',
    r'\bprofile\b'
]



def is_known_section_header(line):
    stripped = line.strip()
    return any(
        re.search(rf'^\s*{pattern}\s*[:\-]?\s*$', stripped, re.IGNORECASE)
        for pattern in CANONICAL_HEADER_PATTERNS
    )


def is_date_like(text):
    return bool(re.match(r'(?i)^\d{1,2}/\d{4}$|^[A-Za-z]{3,9} \d{4}$|^\d{4}$', text.strip()))

def is_location_like(text):
    return bool(re.fullmatch(r'(City|State|[A-Z][a-z]{1,15})', text.strip()))

def is_degree_fragment(text):
    return bool(re.fullmatch(r'[A-Za-z]{2,5}', text.strip()))  # BBA, MSc, etc.


def detect_real_headers(lines):
    section_lines = []

    for i, line in enumerate(lines):
        stripped = line.strip()
        if not stripped:
            continue

        # ✅ Canonical match: skip heuristics & filtering
        if is_known_section_header(stripped):
            section_lines.append((i, stripped))
            continue

        # ❌ Filtering for heuristic-only (to avoid false positives)
        if is_date_like(stripped) or is_location_like(stripped) or is_degree_fragment(stripped):
            continue

    return section_lines





In [ ]:
import regex as re

def preprocess_resume_text(text: str) -> str:
    # Normalize escaped newlines and tabs into actual characters
    text = text.replace('\\n', '\n').replace('\\t', '\t')

    # Clean actual tabs and escaped slashes
    text = text.replace('\\/', '/').replace('\t', ' ')
    # Clean tabs and escaped slashes
    text = text.replace('\\/', '/').replace('\t', ' ')

    # ✅ Normalize: fix spaced slashes in dates (e.g. '08 / 2014' → '08/2014')
    text = re.sub(r'(?<=\d)\s*/\s*(?=\d{4})', '/', text)

    # ✅ Protect date ranges by marking them before splitting on 3+ spaces
    text = re.sub(
        r'(?i)(\d{1,2}/\d{4})\s*(to|-)\s*(\d{1,2}/\d{4}|current|present)',
        r'__DATERANGE__\1 to \3__ENDDATE__',
        text
    )

    # ✅ Collapse "Company — City, State" formatting
    text = re.sub(
        r'(?i)([A-Za-z0-9&.,()\-\' ]{2,})\s*[-－–—]{1,2}\s*([A-Z][a-z]+)\s*,\s*([A-Z][a-z]+)',
        r'\1 — \2 , \3',
        text
    )

    # ✅ Convert 3+ spaces into newlines
    text = re.sub(r' {3,}', '\n', text)

    # Normalize excessive newlines and spaces
    text = re.sub(r'\n{2,}', '\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    # ✅ Restore protected date ranges
    text = text.replace('__DATERANGE__', '\n').replace('__ENDDATE__', '\n')

    # Add breaks around known section headers
    def insert_header_breaks(line):
        stripped = line.strip()
        if is_known_section_header(stripped):
            return f"\n{stripped}\n"
        return stripped

    lines = text.splitlines()
    lines = [insert_header_breaks(line) for line in lines]
    text = "\n".join(lines)

    return text.strip()


In [ ]:
import regex as re

def split_run_on_sentences(text):
    """Split long lines into sentence-like segments while preserving periods."""
    lines = text.splitlines()
    split_lines = []

    for line in lines:
        stripped = line.strip()

        # Skip short or bullet lines
        if not stripped or stripped.startswith('-') or len(stripped) < 50:
            split_lines.append(stripped)
            continue

        # Split after period only if followed by space and capital letter
        if stripped.count('.') >= 2:
            # This keeps the period with the sentence
            segments = re.split(r'(?<=\.) (?=[A-Z])', stripped)
            split_lines.extend([seg.strip() for seg in segments])
        else:
            split_lines.append(stripped)

    return split_lines


In [ ]:
def format_resume_for_llm(text: str) -> str:
    text = preprocess_resume_text(text)
 
    lines = text.splitlines()
     # Sentence splitting
    lines = split_run_on_sentences("\n".join(lines))
  
    headers = detect_real_headers(lines)
    print("Detected Headers:", headers)

    header_indices = {idx for idx, _ in headers}

    output_lines = []
    for i, line in enumerate(lines):
        stripped = line.strip()

        if not stripped:
            continue

        # Skip lines that are isolated numbers (e.g. caused by bad splitting)
        if stripped.isdigit():
            continue

        # Insert a line break before date ranges (if not already done)
        if re.match(r'(?i)^\d{1,2}/\d{4} to (\d{1,2}/\d{4}|current|present)$', stripped):
            output_lines.append("")  # add a break before
            output_lines.append(stripped)
            continue

        # Header detection
        if i in header_indices:
            output_lines.append("")  # add a blank line before header
            output_lines.append(stripped.upper())
            output_lines.append("")  # and after
        else:
            output_lines.append(stripped)

    formatted = "\n".join(output_lines)

    # Normalize inline date ranges (if still embedded)
    formatted = re.sub(
        r'(?i)(\d{1,2}/\d{4})\s*(to|-)\s*(\d{1,2}/\d{4}|current|present)',
        r'\1 to \3',
        formatted
    )

    # Format SKILLS section as bullets
    formatted = re.sub(
        r'(?i)(\nSKILLS\n)([^\n]+)',
        lambda m: m.group(1) + "\n" + "\n".join(f"- {s.strip()}" for s in m.group(2).split(',')),
        formatted
    )

    return formatted.strip()


### Final Orchestrator: Fault-Tolerant Extraction

In [ ]:
import uuid
from typing import Dict, Optional, Type

def inject_ids(parsed: Dict, schema_model: Optional[Type], record_id: str) -> Dict:
    """
    Injects a UUID as `resume_id` or `jd_id` based on the schema model name.
    """
    if not schema_model:
        print("⚠️ No schema model provided for ID injection.")
        return parsed
    schema_name = schema_model.__name__.lower()
    if schema_name.startswith("resume"):
        parsed["resume_id"] = record_id
    elif schema_name.startswith("jobdescription"):
        parsed["jd_id"] = record_id
    return parsed


In [ ]:
from typing import Union

def extract_structured_json(
    text: str,
    prompt_template: str,
    schema_model: Union[None, type] = None,
    max_new_tokens: int = 4096,
    retries: int = 0,
    validate: bool = True,
    record_id: str = "",
    mock_json: str = ""
) -> dict:
    """
    Runs LLM to extract structured JSON and validates against schema.
    Includes: prompt sanitization, retry, echo detection, brace parser fallback, schema validation.
    """
    schema_str = get_schema_str(schema_model)
    prompt = prompt_template.format(text=text, schema=schema_str, mock_json=mock_json)
    raw_output = ""
    attempt = 0

    while attempt <= retries:
        try:
            # Step 1: Get LLM output
            response = generate_llm_output(prompt, max_new_tokens)
            print("🧪 LLM output was:\n", response)  # Preview first 300 chars
            raw_output = sanitize_llm_output(response, prompt)
            #print("🧪 Raw output was:\n", raw_output)

            # Step 2: Detect schema echo or instruction echo
            if "$schema" in raw_output or "Ensure these rules" in raw_output:
                raise ValueError("LLM echoed schema or instruction block instead of generating JSON.")

            # Step 3: Try JSON load directly
            json_start = raw_output.find("{")
            if json_start == -1:
                raise ValueError("No opening '{' found in LLM output.")
            
            cleaned_output = raw_output[json_start:]
            cleaned_output = clean_raw_data(cleaned_output)

            parsed = json.loads(cleaned_output)
            parsed = inject_ids(parsed, schema_model, record_id)
            
            print("🧪 Parsed output was:\n", parsed)  # Preview first 300 chars
            

            # Step 4: Optional schema validation
            if validate and schema_model:
                if hasattr(schema_model, "normalize"):
                    parsed = schema_model.normalize(parsed)
                schema_model.model_validate(parsed)

            return parsed

        except Exception as e:
            print(f"⚠️ Attempt {attempt + 1} failed: {e}")
            #print("🧪 Raw output was:\n", raw_output)  # Preview first 300 chars
            attempt += 1

    # Step 5: Fallback using brace matching
    try:
        parsed = extract_json_block(raw_output)
        parsed = inject_ids(parsed, schema_model, record_id)
            
        if validate and schema_model:
            if hasattr(schema_model, "normalize"):
                parsed = schema_model.normalize(parsed)
            schema_model.model_validate(parsed)
        return parsed
    except Exception as e:
        return {
            "raw_output": raw_output.strip(),
            "error": f"Regex fallback failed: {e}"
        }

In [ ]:
from pydantic import ValidationError

def pydantic_validate(model_class, data):
    """
    Version-safe validator that supports both Pydantic v1 and v2.
    """
    try:
        # Pydantic v2
        return model_class.model_validate(data)
    except AttributeError:
        # Fallback to Pydantic v1
        return model_class.parse_obj(data)


def validate_entry(entry, is_resume):
    try:
        model = ResumeSchema if is_resume else JobDescriptionSchema
        if hasattr(model, "normalize"):
            normalized = model.normalize(entry)
        else:
            normalized = entry
        pydantic_validate(model, normalized)
        return True, None
    except ValidationError as ve:
        return False, str(ve)


##  Normalize in Batches with Validation

In [ ]:
def save_metadata_summary(
    output_dir: Path,
    is_resume: bool,
    input_file: str,
    total_records: int,
    total_valid: int,
    total_invalid: int,
    start_index: int,
    end_index: int,
    timestamp: str,
    batch_id: str
):
    summary = {
        "batch_id": batch_id,
        "timestamp": timestamp,
        "input_file": input_file,
        "input_type": "resume" if is_resume else "job_description",
        "records_start_index": start_index,
        "records_end_index": end_index,
        "records_total": total_records,
        "records_valid": total_valid,
        "records_invalid": total_invalid,
        "output_dir": str(output_dir)
    }
    summary_file = output_dir / f"meta_{'resumes' if is_resume else 'jds'}_{start_index}_{end_index}_{timestamp}_{batch_id}.json"
    save_json_output(summary, str(summary_file), overwrite=True)


In [ ]:
def render_jd_text(record: dict) -> str:
    """
    Constructs a rich text string from all non-empty fields of a JD record.
    """
    jd_parts = []
    for k, v in record.items():
        label = k.replace('_', ' ').title()

        if v is None:
            continue

        if isinstance(v, list):
            if v:
                value_str = ", ".join(str(i) for i in v if i)
                jd_parts.append(f"{label}: {value_str}")
        elif isinstance(v, (str, int, float)):
            value_str = str(v).strip()
            if value_str:
                jd_parts.append(f"{label}: {value_str}")

    return "\n".join(jd_parts)


In [ ]:
import uuid
from datetime import datetime
from typing import List

def normalize_batch(
    records: List[dict],
    start_idx: int,
    end_idx: int,
    is_resume: bool,
    output_dir: Path,
    prompt_template,
    schema_model
):
    results, invalids = [], []

    for record in records:
        if is_resume:
            raw_resume_text  = record.get("Resume_str", "")
            # Step 1: Restore structure
            text = format_resume_for_llm(raw_resume_text)
            #print(f"📝 Normalized input resume text:\n{text} \n")
            max_new_tokens =4096
            record_id = record.get("ID", str(uuid.uuid4()))
            mock_json = get_mock_resume_json()
        else:
            text = render_jd_text(record)
            max_new_tokens = 2048
            record_id = record.get("job_id", str(uuid.uuid4()))
            #mock_json = get_mock_resume_json()

        parsed = extract_structured_json(
            text=text,
            prompt_template=prompt_template,
            schema_model=schema_model,
            max_new_tokens=max_new_tokens,
            record_id = record_id,
            mock_json=mock_json,
            validate=False
        )

        if "raw_output" in parsed or "error" in parsed:
            invalids.append({
                "input": text,
                "output": parsed,
                "error": parsed.get("error", "Malformed or unstructured output")
            })
        else:
            results.append(parsed)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    batch_id = uuid.uuid4().hex[:6]
    prefix = "resumes" if is_resume else "jds"

    if results:
        save_json_output(
            results,
            output_path=output_dir / f"{prefix}_valid_{start_idx}_{end_idx}_{timestamp}_{batch_id}.json"
        )
    if invalids:
        save_json_output(
            invalids,
            output_path=output_dir / f"{prefix}_invalid_{start_idx}_{end_idx}_{timestamp}_{batch_id}.json"
        )

    return results, invalids


In [ ]:
from tqdm import tqdm

def normalize_file_in_batches(
    input_filename: str,
    output_dir: Path,
    is_resume: bool = True,
    input_dir: Path = Path("json_outputs"),
    save_every: int = 5,
    limit: int = None
):
    input_path = input_dir / input_filename
    data = load_ndjson_file(input_path)
    

    checkpoint_file = output_dir / f"checkpoint_{'resumes' if is_resume else 'jds'}.json"
    start_index = 0
    if checkpoint_file.exists():
        checkpoint = load_json_file(checkpoint_file)
        start_index = checkpoint.get("last_index", 0)
        print(f"🔁 Resuming from index {start_index}")

    # Apply limit
    data_to_process = data[start_index:]
    if limit is not None:
        data_to_process = data_to_process[:limit]

    prompt_template = RESUME_PROMPT_TEMPLATE if is_resume else JD_PROMPT_TEMPLATE
    schema_model = ResumeSchema if is_resume else JobDescriptionSchema

    os.makedirs(output_dir, exist_ok=True)

    total_valid, total_invalid = 0, 0
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    batch_id = uuid.uuid4().hex[:6]
    actual_start = start_index
    actual_end = start_index + len(data_to_process)

    for i in tqdm(range(0, len(data_to_process), save_every)):
        batch = data_to_process[i:i + save_every]
        batch_start = start_index + i
        batch_end = batch_start + len(batch)  # ✅ Accurate

        results, invalids = normalize_batch(
            records=batch,
            start_idx=batch_start,
            end_idx=batch_end,
            is_resume=is_resume,
            output_dir=output_dir,
            prompt_template=prompt_template,
            schema_model=schema_model
        )

        total_valid += len(results)
        total_invalid += len(invalids)

        save_json_output({"last_index": batch_end}, str(checkpoint_file), overwrite=True)

    # ✅ Save metadata summary
    """
    save_metadata_summary(
        output_dir=output_dir,
        is_resume=is_resume,
        input_file=input_filename,
        total_records=len(data_to_process),
        total_valid=total_valid,
        total_invalid=total_invalid,
        start_index=actual_start,
        end_index=actual_end,
        timestamp=timestamp,
        batch_id=batch_id
    )
    """


## Run Phase 2 End-to-End

In [ ]:
normalize_file_in_batches(
    input_filename="parsed_jds.json",
    input_dir=Path(Config.JSON_OUTPUT_DIR),
    output_dir=Path(Config.JSON_OUTPUT_NORMALIZED_DIR),
    is_resume=False,
    save_every=5,
    limit=60  # ✅ Process only 20 records max
)


In [ ]:
normalize_file_in_batches(
    input_filename="parsed_resumes.json",
    input_dir=Path(Config.JSON_OUTPUT_DIR),
    output_dir=Path(Config.JSON_OUTPUT_NORMALIZED_DIR),
    is_resume=True,
    save_every=1,
    limit=1  # ✅ Process only 20 records max
)


## Merge normalized files

In [ ]:
# Paths
normalized_dir = Path(Config.JSON_OUTPUT_NORMALIZED_DIR)
merged_dir = normalized_dir / "merged"

merge_json_files(
    source_dir=normalized_dir,
    output_file=normalized_dir / "normalized_jds.json",
    pattern="jds_valid*.json",
    merged_dir=merged_dir
)

merge_json_files(
    source_dir=normalized_dir,
    output_file=normalized_dir / "normalized_resumes.json",
    pattern="resumes_valid*.json",
    merged_dir=merged_dir
)
